In [1]:
import string
import nltk
import enchant
from spello.model import SpellCorrectionModel
import textstat
import re

nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lbiondo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lbiondo\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
#load spell checker model
sp = SpellCorrectionModel(language='en')
sp.load('./spello_model/en_large.pkl')

c:\Users\lbiondo\Anaconda3\envs\tesi\lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was saved on spell<1.3.0. As such due to a bug in previous versions, "


In [3]:
def removePunctuationFromTokenized(contentsTokenized):
    excludePuncuation = set(string.punctuation)
    
    # manually add additional punctuation to remove
    doubleSingleQuote = '\'\''
    doubleDash = '--'
    doubleTick = '``'

    excludePuncuation.add(doubleSingleQuote)
    excludePuncuation.add(doubleDash)
    excludePuncuation.add(doubleTick)

    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]
    return filteredContents

#### Lexical Diversity

In [4]:
def lexical_diversity(text):
   return (len(set(text)) / len(text))

#### Percentage of Uppercased and Lowercased Words 

In [5]:
def percentage_uppercased(text_tokenized):
    islower = 0;
    isupper = 0;
    total_words = len(text_tokenized);

    for word in text_tokenized:      
        if word.islower():
            islower +=1         
        elif word.isupper():
            isupper+=1         

    return (islower/total_words,isupper/total_words)

#### Spell Checking Against Dictionary

In [6]:
def spell_checker_1(text_tokenized):
    d = enchant.Dict("en_US")
    mistakes = 0
    for word in text_tokenized:
        if not d.check(word):
            mistakes+=1
    return mistakes        

#### Spell Checking with ML model

In [7]:
def spell_checker_2(raw_text,text_tokenized):
    corrected = sp.spell_correct(raw_text)
    i = 0
    for w in text_tokenized:
        if(w in corrected['correction_dict']):
            i+=1
    return i

#### Unrecognized Words by POS Tagger

In [8]:
def unrecognized_by_pos(text_tokenized):
    text_tagged = nltk.pos_tag(text_tokenized,tagset='universal')
    unknown = 0
    for t in text_tagged:
         if t[1] == "X":
                #return t
                unknown += 1
    return unknown/len(text_tagged)            

#### Average Sentence Length

In [9]:
def avg_sentence_length(text):
    
    terminating_punct = "[!?.]"
    punct = r"\W"  # non-word characters
    sentences = [
        s.strip()  # without trailing whitespace
        for s in re.split(
            terminating_punct,
            "".join(text).replace("\n", " "),  # text as 1 string
        )
        if s.strip()  # non-empty
    ]

    def wordcount(s):
        """Split sentence s on punctuation
        and return number of non-empty words
        """
        return len([w for w in re.split(punct, s) if w])
    #map each sentece to its wordcount then sum all the wordcounts
    return sum(map(wordcount, sentences)) / len(sentences)


#### Acronyms 1

In [10]:
#this finds acronyms with with uppercase charcters with numbers inside
def acronym1(s):
    return re.findall(r"\b(?:[0-9]+[A-Z][A-Z0-9]*)|(?:[A-Z][A-Z0-9]+)\b", s)

#### Acronyms 2

In [11]:
#this finds acronyms of all letters in uppercase and all letters in uppercase followed by dots 
def acronym2(s):
    return re.findall(r"\b[A-Z\.]{2,}\b", s)

#### Acronyms

In [12]:
def acronym(s):
    acronym1 = re.findall(r"\b(?:[0-9]+[A-Z][A-Z0-9]*)|(?:[A-Z][A-Z0-9]+)\b|\b[A-Z\.]{2,}\b", s)
    acronym2 = re.findall(r"\b[A-Z\.]{2,}\b", s)
    return  acronym1


In [13]:
path = "./data/input/input.txt"

with open(path, "r") as ifile:
    raw_text = ifile.read()
    
#raw_text = "Sample text to try functions"    
text_tokenized = removePunctuationFromTokenized(nltk.word_tokenize(raw_text))
lower,upper = percentage_uppercased(text_tokenized)
print("Lexical diversity: " + "{0:.3f}".format(lexical_diversity(text_tokenized)))
print("Lowercase sentences: "+ "{0:.3f}".format(lower))
print("Uppercase sentences: "+ "{0:.3f}".format(upper))
print("Spelling mistakes 1: "  + str(spell_checker_1(text_tokenized)))
print("Spelling mistakes 2: "  + str(spell_checker_2(raw_text,text_tokenized)))
print("Unrecognized by POS tagger: "+str(unrecognized_by_pos(text_tokenized)))
print("Average sentence length: " +"{0:.3f}".format(avg_sentence_length(raw_text)))
print("Readability (CLI): " + str(textstat.coleman_liau_index(raw_text)))
print("Readability (ARI): " + str(textstat.automated_readability_index(raw_text)))
print("Acronyms:" + str(acronym(raw_text)))

Lexical diversity: 0.957
Lowercase sentences: 0.739
Uppercase sentences: 0.174
Spelling mistakes 1: 14
Spelling mistakes 2: 7
Unrecognized by POS tagger: 0.043478260869565216
Average sentence length: 6.000
Readability (CLI): 8.27
Readability (ARI): 7.3
Acronyms:['ABC', 'L.B', 'EC2', 'ABC']
